In [1]:
import os
import sys

import numpy as np
import pandas as pd
from datetime import datetime
import pdb
import xarray as xr
import matplotlib.pyplot as plt
import puv

import solo

sys.path.append(r'c:\checkouts\python\TUD-COASTAL\python')

In [2]:
dataFolder="ps_data/tailored_with_c/"

ds1 = xr.open_dataset(dataFolder+"ps1.nc")
ds2 = xr.open_dataset(dataFolder+"ps2.nc")
ds3 = xr.open_dataset(dataFolder+"ps3.nc")
ds4 = xr.open_dataset(dataFolder+"ps4.nc")
ds5 = xr.open_dataset(dataFolder+"ps5.nc")
ds6 = xr.open_dataset(dataFolder+"ps6.nc")

display(ds1)

<xarray.Dataset>
Dimensions:         (f: 128, t: 245)
Coordinates:
  * f               (f) float64 0.0 0.03125 0.0625 0.09375 ... 3.906 3.938 3.969
  * t               (t) datetime64[ns] 2023-09-23T10:09:32 ... 2023-09-29T11:...
Data variables: (12/24)
    zi              (t) float64 ...
    zb              (t) float64 ...
    sf              float64 ...
    name            object ...
    h               (t) float64 ...
    vy              (t, f) float64 ...
    ...              ...
    p_sig           (t) float64 ...
    vel_Sk          (t) float64 ...
    vel_As          (t) float64 ...
    vel_sig         (t) float64 ...
    epsilon         (t) float64 ...
    cb_10           (t) float64 ...
Attributes: (12/16)
    Conventions:               CF-1.6
    name:                      ps1
    instrument:                ps1
    instrument type:           Ruskin RBR Solo
    instrument serial number:  202438
    epsg:                      28992
    ...                        ...
    summary:                   Hydraulic fieldwork pressure data, with pressu...
    contact person:            Marlies van der Lugt
    emailadres:                m.a.vanderlugt@tudelft.nl
    construction datetime:     19-Oct-2023 (20:25:02)
    version:                   v1
    version comments:          constructed with xarray

In [18]:
# list_of_ds = [ds1, ds2, ds3, ds4, ds5, ds6]

# def get_location_of_breaking(list_of_ds):
    
    
#     for i in range(len(list_of_ds) - 1):
        
#         list_of_ds[i].Hm0.values

energy_loss1 = []
energy_loss2 = []
energy_loss3 = []
energy_loss4 = []
energy_loss5 = []

t_data = []

for t in ds6.t.values:
    
    Hm0_6 = ds6.sel(t=t).Hm0.values
    
    def get_Hm0(ds, t):
    
        try:
            Hm0 = ds.sel(t=t).Hm0.values
        except KeyError:
            Hm0 = np.NAN
        return Hm0
    
    Hm0_1 = get_Hm0(ds1, t)
    Hm0_2 = get_Hm0(ds2, t)
    Hm0_3 = get_Hm0(ds3, t)
    Hm0_4 = get_Hm0(ds4, t)
    Hm0_5 = get_Hm0(ds5, t)

    energy_loss1.append(Hm0_2 - Hm0_1)
    energy_loss2.append(Hm0_3 - Hm0_2)
    energy_loss3.append(Hm0_4 - Hm0_3)
    energy_loss4.append(Hm0_5 - Hm0_4)
    energy_loss5.append(Hm0_6 - Hm0_5)
    
    t_data.append(t)


In [33]:
df = pd.DataFrame(columns=['t', 'E1', 'E2', 'E3', 'E4', 'E5'])

df.t = t_data
df.E1 = energy_loss1
df.E2 = energy_loss2
df.E3 = energy_loss3
df.E4 = energy_loss4
df.E5 = energy_loss5

display(df)

,t,E1,E2,E3,E4,E5
0,2023-09-23 15:00:00,NaN,NaN,NaN,NaN,NaN
1,2023-09-23 15:10:00,NaN,NaN,NaN,NaN,0.446629
2,2023-09-23 15:20:00,NaN,NaN,NaN,NaN,0.334096
3,2023-09-23 15:30:00,NaN,NaN,NaN,NaN,0.366050
4,2023-09-23 15:40:00,NaN,NaN,NaN,NaN,0.318147
...,...,...,...,...,...,...
837,2023-09-29 10:30:00,NaN,NaN,NaN,-0.062427,0.270740
838,2023-09-29 10:40:00,NaN,NaN,NaN,NaN,NaN
839,2023-09-29 10:50:00,NaN,NaN,NaN,NaN,NaN
840,2023-09-29 11:00:00,NaN,0.011556,0.002232,0.000639,-0.003126


In [42]:
loss_threshold = 0.05

breaker_location = np.zeros(len(df))

for i in df.index:
    
    if df.loc[i].E5 > loss_threshold:
        breaker_location[i] = 5
        
    elif df.loc[i].E4 > loss_threshold:
        breaker_location[i] = 4
        
    elif df.loc[i].E3 > loss_threshold:
        breaker_location[i] = 3
        
    elif df.loc[i].E2 > loss_threshold:
        breaker_location[i] = 2
        
    elif df.loc[i].E1 > loss_threshold:
        breaker_location[i] = 1
        
    else:
        breaker_location[i] = 0
    

[0. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 2. 2. 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 3. 3. 3. 3. 4. 4. 4. 3. 4. 4. 5. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 3. 4. 4. 4. 4.
 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 0. 5. 0. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 4. 4. 4. 4. 3. 4. 3. 3. 3. 3. 3. 3. 2. 0. 0. 0.
 0. 0. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 4. 4. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4. 4. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 0. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 0. 0. 4. 4. 4. 4. 4. 3. 3.

In [47]:
df_new = pd.DataFrame(columns=['t', 'breaker_location'])

df_new.t = t_data
df_new.breaker_location = breaker_location

df_new.to_csv('breaker_location.csv')